In [10]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy import signal
import tqdm
import multiprocessing as mp

1.1

In [21]:
def kuramoto(w, N = 500, K = 1.5, t_max = 60.0, d_t = 0.01, sigma = 1.0):
    #timestamps
    T = np.arange(0, t_max, d_t)
    #array of phases
    theta = np.zeros((N, len(T)))
    #initial states
    for i in range(N):
        theta[i][0] = np.random.uniform(-np.pi, np.pi)
    #computation of changes in phases
    for k in range(1, len(T)):
        for i in range(N):
            temp = np.zeros((N))
            for j in range(N):
                temp[i] = np.sin(theta[i][k-1] - theta[j][k-1])
            theta[i][k] = theta[i][k-1] + (2 * np.pi * w[i] + K / N * np.sum(temp) + np.random.normal(0.0, float(sigma))) * d_t
    #array of model phases
    phase_signal = np.zeros(len(T))
    #array of model signals
    signal = np.zeros(len(T))
    #computation of model signals
    for i in range(len(T)):
        ps = np.average(theta[:,i])
        phase_signal[i] = ps
        signal[i] = np.exp(complex(0, ps))
    #PSD
    freqs, psd = sp.signal.welch(signal, fs=int(1/d_t), scaling="spectrum")
    #return
    return(psd, freqs, signal, T)

1.2

In [12]:
freqs = [10]*10 + [15]*5 + [25]*5

In [13]:
def w_gen(freq): 
    w = np.zeros((500))
    for i in range(500):
        w[i] = np.random.uniform(0.75 * freq, 1.25 * freq)
    return(w)

W = np.zeros((500, len(freqs)))
for i in range(len(freqs)):
    w = w_gen(freqs[i])
    W[:,i] = w

In [14]:
s_psd = []
s_freqs = []
for freq in tqdm.tqdm(freqs):
    psd, freqs, signal, T = kuramoto(w_gen(freq))
    s_psd.append(psd)
    s_freqs.append(freqs)
    
avg_psd = np.mean(s_psd, axis = 0)
avg_freqs = np.mean(s_freqs, axis = 0) 

  5%|████                                                                            | 1/20 [09:35<3:02:22, 575.93s/it]


KeyboardInterrupt: 